In [ ]:
# This command installs the ‘requests’ library if it is not already present.
# Installs the ‘requests’ library, which is needed to make HTTP requests to the ElabFTW API.
!pip install requests

In [ ]:
# Import the ‘requests’ library to make HTTP requests (GET, POST, etc.).
# Import the ‘json’ module to handle data in JSON format.
# Useful for constructing or reading data in JSON format.
import requests
import json

# Definition of a class to interact with the ElabFTW API.
class ElabFTWAPI:
# Constructor method of the class: executed when creating an instance of the class.
# Creation of a custom client for ElabFTW, encapsulating the authentication and URL base.
    def __init__(self, base_url, api_key):
        """
        Initializes the ElabFTW API client.

        Args:
            base_url: The base URL of the ElabFTW instance.
            api_key: Your API key for authentication.
        """

        self.base_url = base_url
        self.headers = {
            'Authorization': f'{api_key}', # Add authentication token (APIkey)
            'Content-Type': 'application/json' # Indicates that the request body is in JSON format
        }

# GET request to the ElabFTW API, to obtain data on experiments.
    def get_experiments(self, **kwargs):
        """
        Retrieves a list of experiments from ElabFTW.

        Args:
            **kwargs: Optional parameters for filtering the experiments.
                      Refer to the ElabFTW API documentation for available filters.

        Returns:
            A list of experiments in JSON format.
            The API response will contain the requested information about the experiments.
        """

        url = f'{self.base_url}/api/v2/experiments'
        response = requests.get(url, headers=self.headers, params=kwargs)
        self._check_response(response)
        return response.json()


    def get_experiment(self, experiment_id):
        """
        Retrieves details for a specific experiment.

        Args:
            experiment_id: The ID of the experiment to retrieve.

        Returns:
            The experiment details in JSON format.
        """
# Constructs the full URL of the API to retrieve the details of a specific experiment
# Use the experiment ID to create the endpoint
        url = f'{self.base_url}/api/v2/experiments/{experiment_id}'
        response = requests.get(url, headers=self.headers)
        self._check_response(response)
        return response.json()

# This is a class method that retrieves a list of ‘objects’ (items) from ElabFTW.
    def get_items(self, **kwargs):
        """
        Retrieves a list of items from ElabFTW.

        Args:
            **kwargs: Optional parameters for filtering the items.
                      Refer to the ElabFTW API documentation for available filters.

        Returns:
            A list of items in JSON format.
        """
# Constructs the URL to make a GET request to the ElabFTW API to retrieve objects (items).
        url = f'{self.base_url}/api/v2/items'
        response = requests.get(url, headers=self.headers, params=kwargs)
        self._check_response(response)
        return response.json()

# The class method that retrieves the details of a specific object (item).The ID of the item to retrieve is item_id.
    def get_item(self, item_id):
        """
        Retrieves details for a specific item.

        Args:
            item_id: The ID of the item to retrieve.

        Returns:
            The item details in JSON format.
        """
# Constructs the full URL to make a GET request to the ElabFTW API
# The object ID (item_id) is inserted into the endpoint to retrieve a specific object (item)
        url = f'{self.base_url}/api/v2/items/{item_id}'
        response = requests.get(url, headers=self.headers)
        self._check_response(response)
        return response.json()

# This is a class method that retrieves the tags associated with a specific experiment.
# The experiment_id parameter is the unique ID of the experiment, which is passed to the method to obtain the tags for that specific experiment.
    def get_experiment_tags(self, experiment_id):

        url = f'{self.base_url}/api/v2/experiments/{experiment_id}/tags'
        response = requests.get(url, headers=self.headers)
        self._check_response(response)
        return response.json()

# Create a new experiment in ElabFTW, taking the experiment data (data) and the list of tags to add (tags)
    def create_experiment(self, data, tags):
        """
        Creates a new experiment by first posting an empty record,
        then updating it via PATCH with the provided data.

        Args:
            data: A dictionary containing the experiment data.
            tags: A list of tags to associate with the experiment.
        Returns:
            The updated experiment data in JSON format.
        """
        # Step 1: Create empty experiment
        url = f'{self.base_url}/api/v2/experiments'
        response = requests.post(url, headers=self.headers)
        self._check_response(response)

        # Step 2: Extract ID from Location header.The ‘Location’ header contains the URL of the newly created object. It is used to extract the experiment ID.
        location = response.headers.get('Location')

        if not location or not location.rsplit('/', 1)[-1].isdigit():
            raise Exception('Failed to retrieve new experiment ID from Location header.')

        experiment_id = location.rsplit('/', 1)[-1]


        # Step 3: Patch the experiment with actual data
        patch_url = f'{url}/{experiment_id}'

        # Send a PATCH request to the API to update the experiment data
        # The request will include the data to be updated (contained in the variable ‘data’)
        patch_response = requests.patch(patch_url, headers=self.headers, json=data)


        self._check_response(patch_response)
        url = f'{self.base_url}/api/v2/experiments/{experiment_id}/tags'

        # For each tag in the provided tag list, send a POST request to add the tag to the experiment
        for tag in tags:
          data={"tag":tag}
          print(data)
          response = requests.post(url, headers=self.headers,json=data)
          self._check_response(response)
        return patch_response.json()


# Create a new object (item) on ElabFTW, taking the object data (data) and the list of tags to be assigned to that object (tags)
    def create_item(self, data, tags):
        """
        Creates a new item by first posting an empty record,
        then updating it via PATCH with the provided data.

        Args:
            data: A dictionary containing the item data.
            tags: A list of tags to associate with the item.

        Returns:
            The updated item data in JSON format.
        """
        # Step 1: Create empty item
        url = f'{self.base_url}/api/v2/items' # Build the URL to create a new item.
        response = requests.post(url, headers=self.headers)
        self._check_response(response)

        # Step 2: Extract ID from Location header
        location = response.headers.get('Location')
        if not location or not location.rsplit('/', 1)[-1].isdigit():
            raise Exception('Failed to retrieve new item ID from Location header.')

        item_id = location.rsplit('/', 1)[-1]

        # Step 3: Patch the items with actual data
        patch_url = f'{url}/{item_id}'
        patch_response = requests.patch(patch_url, headers=self.headers, json=data)
        self._check_response(patch_response)
        url = f'{self.base_url}/api/v2/items/{item_id}/tags'
        for tag in tags:
          data={"tag":tag}
          print(data)
          response = requests.post(url, headers=self.headers,json=data)
          self._check_response(response)
        return patch_response.json()
        return response.json()

# To retrieve the list of API keys associated with the user or system on ElabFTW
    def get_apikeys(self):
        url = f'{self.base_url}/api/v2/apikeys'
        response = requests.get(url, headers=self.headers)
        self._check_response(response)
        return response.json()

# Update an existing experiment on ElabFTW, using its ID (experiment_id) and updated data (data).
    def update_experiment(self, experiment_id, data):
        """
        STUB
        Updates an existing experiment.

        Args:
          experiment_id: The ID of the experiment to update.
          data: A dictionary containing the updated experiment data.

        Returns:
            The updated experiment data in JSON format.
        """

# Error management.
    def _check_response(self, response):
        """
        Checks the HTTP response for errors.

        Args:
            response: The HTTP response object.

        Raises:
            Exception: If the response indicates an error.
        """
        if not 200 <= response.status_code < 300:


                try:
                  print(response)
                  print(response.text)

                  error_data = response.json()
                  error_message = error_data.get('message', 'Unknown error')
                except json.JSONDecodeError:
                  error_message = response.text
                raise Exception(f'ElabFTW API request failed with status code {response.status_code}: {error_message}')



In [ ]:
base_url = 'https://prp-electronic-lab.areasciencepark.it'
api_key = ''
api_key_test = ''
elabftw_api = ElabFTWAPI(base_url, api_key)

In [ ]:
elabftw_api.get_item(49) # GET request to retrieve information of a single "item" from ElabFTW in JSON format

{'access_key': None,
 'available': 1,
 'body': '<p><strong><span style="font-size:14pt;">Analysis:</span></strong></p>\n<p>A more in-depth investigation was conducted on Sample 3 using SI-STEM and HRTEM techniques.</p>\n<p>The HAADF image displays intensity variations corresponding to different atomic densities, effectively highlighting the contrast and clearly revealing the presence of carbon dots. The elemental maps (carbon map and nitrogen map) further confirm the co-existence of carbon and nitrogen within the sample. The uniform distribution of nitrogen throughout the carbon matrix, in the elemental maps, suggests successful nitrogen doping. The incorporation of nitrogen changes the characteristics of carbon nanoparticles, potentially improving the electronic properties and stability of the material.</p>\n<p>The HRTEM image reveals lattice fringes indicative of a partially ordered structure.\xa0</p>\n<p>The Fourier transform analysis displays distinct intensity distributions along 

In [ ]:
# BUILD EXPERIMENT PAYLOAD
def build_experiment_payload(
    title: str,
    body_html: str,
    date: str,
    category: int,
    status: int,
    canread: dict = None,
    canwrite: dict = None,
    metadata: dict = None
) -> dict:
    """
    Build a payload dictionary to update an ElabFTW experiment.

    Args:
        title: The title of the experiment.
        body_html: The HTML-formatted body of the experiment.
        date: The experiment date in 'YYYY-MM-DD' format.
        tags: A list of tag strings.
        category: (optional) The category ID of the experiment.
        status: (optional) The status ID of the experiment.
        metadata: (optional) The custom metadata to associate with an experiment on ElabFTW
    Returns:
        A dictionary ready to be sent in a PATCH request to ElabFTW.
    """
    payload = {
        'title': title,
        'body': body_html,
        'date': date,
        'category': category,
        'status': status,
        'metadata': metadata
    }

    return payload

# BUILD ITEM PAYLOAD
def build_item_payload(
    title: str,
    body_html: str,
    date: str,
    category: int,
    status: int,
    canread: dict = None,
    canwrite: dict = None,
    metadata: dict = None
) -> dict:
    """
    Build a payload dictionary to update an ElabFTW item.

    Args:
        title: The title of the item.
        body_html: The HTML-formatted body of the item.
        date: The experiment date in 'YYYY-MM-DD' format.
        tags: A list of tag strings.
        category: The category ID of the item.
        status: The status ID of the item.
        metadata: (optional) The custom metadata to associate with an item on ElabFTW

    Returns:
        A dictionary ready to be sent in a PATCH request to ElabFTW.
    """
    payload = {
        'title': title,
        'body': body_html,
        'date': date,
        'category': category,
        'status': status,
        'metadata': metadata
    }

    return payload


def generate_body_html(**sections) -> str:
    """
    Generate HTML-formatted body for an ElabFTW experiment.

    Accepts dynamic keyword arguments where each key is the section title,
    and the value is the section content.

    Example:
        generate_body_html(
            Goal="Test synthesis of carbon nanotubes.",
            Synthesis_Parameters="250°C, 2 hours, basic pH.",
            Results="Fluorescence detected under IR light."
        )
    """
# Create an empty list that will contain each HTML section (as separate strings), which will then be joined.
    html_parts = []
    for section_title, content in sections.items():
        clean_title = section_title.replace("_", " ").capitalize()
        html_parts.append(f"<p><strong>{clean_title}:</strong> {content}</p>")
    return "\n".join(html_parts)

def generate_html_table(data: dict, title: str = None) -> str:
    """
    Generate an HTML table from a dictionary.

    Args:
        data: A dictionary where keys are row labels and values are row contents (strings or numbers).
        title: Optional title displayed above the table.

    Returns:
        HTML string representing the table.
    """
    html = ""

# If a title is provided, adds an HTML paragraph with the title above the table.
    if title:
        html += f"<p><strong>{title}</strong></p>\n"

    html += "<table border='1' cellspacing='0' cellpadding='5'>"
    html += "<thead><tr><th>Parameter</th><th>Value</th></tr></thead>"
    html += "<tbody>"

    for key, value in data.items():
        html += f"<tr><td>{key}</td><td>{value}</td></tr>"

    html += "</tbody></table>"
    return html






In [ ]:
# Editable function of this json object: "'''{
#     "elabftw": {
#         "extra_fields_groups": [
#             {
#                 "id": 1,
#                 "name": "Instrument metadata"
#             }
#         ]
#     },
#     "extra_fields": {
#         "Camera": {
#             "type": "radio",
#             "value": "{camera_type}",
#             "options": [
#                 "Rio 16",
#                 "K3",
#                 "Continuum"
#             ],
#             "group_id": 1,
#             "position": 0,
#             "required": true
#         },
#         "Magnification": {
#             "type": "text",
#             "value": "1",
#             "group_id": 1,
#             "position": 2,
#             "required": true
#         },
#         "Exposure time (s)": {
#             "type": "text",
#             "value": "1",
#             "group_id": 1,
#             "position": 3,
#             "required": true
#         },
#         "Applied Voltage (keV)": {
#             "type": "text",
#             "value": "1",
#             "group_id": 1,
#             "position": 4,
#             "required": true
#         }
#     }
# }'''"

import json

def edit_json_object(json_data, updates):
    """
    Edits a JSON object based on provided updates.

    Args:
        json_data: The JSON object (dictionary) to be modified.
        updates: A dictionary where keys represent the paths to the elements
            in json_data and values are the new values.  Paths should use
            dot notation, e.g., "elabftw.extra_fields_groups.0.name".

    Returns:
        The updated JSON object (dictionary).  Returns the original object
        if no updates were provided.
    """

    if not updates:  # If there are no updates, the function returns the original dictionary without modifications.
        return json_data

    json_copy = json_data.copy()

    for path, new_value in updates.items():
        parts = path.split(".")
        current = json_copy
        for i, part in enumerate(parts[:-1]):
            try:
                if part.isdigit():
                    part = int(part)
                current = current[part]
            except (KeyError, IndexError, TypeError):
                print(f"Warning: Invalid path segment '{part}' in '{path}'. Skipping update.")
                break
        else:
            try:
                if parts[-1].isdigit():
                    parts[-1] = int(parts[-1])
                current[parts[-1]] = new_value
            except (KeyError, IndexError, TypeError):
                print(f"Warning: Could not update value at path '{path}'.")

    return json_copy


# EXAMPLE USAGE:
json_string = """
{
    "elabftw": {
        "extra_fields_groups": [
            {
                "id": 1,
                "name": "Instrument metadata"
            }
        ]
    },
    "extra_fields": {
        "Camera": {
            "type": "radio",
            "value": "{camera_type}",
            "options": [
                "Rio 16",
                "K3",
                "Continuum"
            ],
            "group_id": 1,
            "position": 0,
            "required": true
        },
        "Magnification": {
            "type": "text",
            "value": "",
            "group_id": 1,
            "position": 2,
            "required": true
        },
        "Exposure time (s)": {
            "type": "text",
            "value": "",
            "group_id": 1,
            "position": 3,
            "required": true
        },
        "Applied Voltage (keV)": {
            "type": "text",
            "value": "",
            "group_id": 1,
            "position": 4,
            "required": true
        }
    }
}
"""


json_object = json.loads(json_string) # Parse the JSON string into a Python dictionary (JSON object)

updates = {
    "elabftw.extra_fields_groups.0.name": "New Instrument Metadata",
    "extra_fields.Camera.value": "K3",
    "extra_fields.Magnification.value": "100",
    "extra_fields.Exposure time (s).value": "30",
    "extra_fields.Applied voltage (keV).value": "200",
    "extra_fields.NonExistentField.value": "newvalue"
}


updated_json = edit_json_object(json_object, updates) # Call the 'edit_json_object' function to apply the updates to the JSON object.

print(json.dumps(updated_json, indent=2))  # Convert the updated JSON object back to a string with pretty formatting and print it.


{
  "elabftw": {
    "extra_fields_groups": [
      {
        "id": 1,
        "name": "New Instrument Metadata"
      }
    ]
  },
  "extra_fields": {
    "Camera": {
      "type": "radio",
      "value": "K3",
      "options": [
        "Rio 16",
        "K3",
        "Continuum"
      ],
      "group_id": 1,
      "position": 0,
      "required": true
    },
    "Magnification": {
      "type": "text",
      "value": "100",
      "group_id": 1,
      "position": 2,
      "required": true
    },
    "Exposure time (s)": {
      "type": "text",
      "value": "30",
      "group_id": 1,
      "position": 3,
      "required": true
    },
    "Applied Voltage (keV)": {
      "type": "text",
      "value": "",
      "group_id": 1,
      "position": 4,
      "required": true
    }
  }
}


In [ ]:
import copy # Import the copy module to create deep copies of objects.
import json # Import the json module for converting data to/from JSON format.

class InstrumentMetadata:
    def __init__(self, data):
        self._original = copy.deepcopy(data)
        ef = data.get("extra_fields", {})

        # Extract editable value fields from 'extra_fields' for later modification.
        self.camera = ef.get("Camera", {}).get("value", "")
        self.magnification = ef.get("Magnification", {}).get("value", "")
        self.exposure_time = ef.get("Exposure time (s)", {}).get("value", "")
        self.applied_voltage = ef.get("Applied Voltage (keV)", {}).get("value", "")


    def to_json(self):

        updated = copy.deepcopy(self._original)

       # Update the "value" fields in the 'extra_fields' dictionary with the edited values.
        updated["extra_fields"]["Camera"]["value"] = self.camera
        updated["extra_fields"]["Magnification"]["value"] = self.magnification
        updated["extra_fields"]["Exposure time (s)"]["value"] = self.exposure_time
        updated["extra_fields"]["Applied Voltage (keV)"]["value"] = self.applied_voltage

        return updated

    def __str__(self):
        return json.dumps(self.to_json(), indent=2)


input_metadata = '''{
    "elabftw": {
        "extra_fields_groups": [
            {
                "id": 1,
                "name": "Instrument metadata"
            }
        ]
    },
    "extra_fields": {
        "Camera": {
            "type": "radio",
            "value": "{camera_type}",
            "options": [
                "Rio 16",
                "K3",
                "Continuum"
            ],
            "group_id": 1,
            "position": 0,
            "required": true
        },
        "Magnification": {
            "type": "text",
            "value": "1",
            "group_id": 1,
            "position": 2,
            "required": true
        },
        "Exposure time (s)": {
            "type": "text",
            "value": "1",
            "group_id": 1,
            "position": 3,
            "required": true
        },
        "Applied Voltage (keV)": {
            "type": "text",
            "value": "1",
            "group_id": 1,
            "position": 4,
            "required": true
        }
    }
}'''


In [ ]:
fixed_field_metadata = InstrumentMetadata(json.loads(input_metadata))  # Create an instance of InstrumentMetadata by parsing the input_metadata JSON string.

# Edit values
fixed_field_metadata.camera = "K3"
fixed_field_metadata.magnification = "100x"
fixed_field_metadata.exposure_time = "3s"
fixed_field_metadata.applied_voltage = "80"


print(json.dumps(fixed_field_metadata.to_json()))

{"elabftw": {"extra_fields_groups": [{"id": 1, "name": "Instrument metadata"}]}, "extra_fields": {"Camera": {"type": "radio", "value": "K3", "options": ["Rio 16", "K3", "Continuum"], "group_id": 1, "position": 0, "required": true}, "Magnification": {"type": "text", "value": "100x", "group_id": 1, "position": 2, "required": true}, "Exposure time (s)": {"type": "text", "value": "3s", "group_id": 1, "position": 3, "required": true}, "Applied Voltage (keV)": {"type": "text", "value": "80", "group_id": 1, "position": 4, "required": true}}}


In [ ]:
# EXAMPLE USAGE FOR TABLE OF METADATA IN MAIN TEXT:

body_html = generate_body_html(Goal=" Optimized Synthesis and Characterization of Highly Reproducible Carbon Dots for Bioimaging Applications", Synthesis_Parameters= "Urea/Citric Acid Ratio about the 4 Samples: 0.6(sample001), 0.8(sample002), 1.0(sample003), 1.2(sample004)", Results="Mixed-size nanocrystals.Miller indices of max. diffraction reflection: (002), (100), (102).= GRAFITIC STRUCTURE", )
metadata = { "Pixel size":0 , "Pixel depht":0 , "Stage alpha":0 , "Stage beta":0 , "Stage x":0 , "Stage y":0 , "Stage z":0 , "Active size":0 }
body_html += generate_html_table(metadata, title="Metadata")
print(body_html)

fixed_field_metadata = InstrumentMetadata(json.loads(input_metadata))

# Edit values
fixed_field_metadata.camera = "K3"
fixed_field_metadata.magnification = "100x"
fixed_field_metadata.exposure_time = "3s"
fixed_field_metadata.applied_voltage = "80"

# Get the updated JSON
updated_data = fixed_field_metadata.to_json()


# EXAMPLE USAGE OF CREATE EXPERIMENT (replace with your actual data)

new_experiment_data = build_experiment_payload( title='Fluorescent Carbon Dots',
    body_html=body_html,
    date='2025-03-23',
    category=2,
    status=2,
    metadata= json.dumps(updated_data)
)
# print(new_experiment_data)



new_experiment = elabftw_api.create_experiment(new_experiment_data,tags=['electronmicroscopy','tem','hrtem', 'carbondots', 'nanoparticles', 'characterization'])
print(json.dumps(new_experiment, indent=2))


<p><strong>Goal:</strong>  Optimized Synthesis and Characterization of Highly Reproducible Carbon Dots for Bioimaging Applications</p>
<p><strong>Synthesis parameters:</strong> Urea/Citric Acid Ratio about the 4 Samples: 0.6(sample001), 0.8(sample002), 1.0(sample003), 1.2(sample004)</p>
<p><strong>Results:</strong> Mixed-size nanocrystals.Miller indices of max. diffraction reflection: (002), (100), (102).= GRAFITIC STRUCTURE</p><p><strong>Metadata</strong></p>
<table border='1' cellspacing='0' cellpadding='5'><thead><tr><th>Parameter</th><th>Value</th></tr></thead><tbody><tr><td>Pixel size</td><td>0</td></tr><tr><td>Pixel depht</td><td>0</td></tr><tr><td>Stage alpha</td><td>0</td></tr><tr><td>Stage beta</td><td>0</td></tr><tr><td>Stage x</td><td>0</td></tr><tr><td>Stage y</td><td>0</td></tr><tr><td>Stage z</td><td>0</td></tr><tr><td>Active size</td><td>0</td></tr></tbody></table>
{'tag': 'electronmicroscopy'}
{'tag': 'tem'}
{'tag': 'hrtem'}
{'tag': 'carbondots'}
{'tag': 'nanoparticles'

FINE CREAZIONE EXPERIMENT


In [ ]:
base_url = 'https://prp-electronic-lab.areasciencepark.it'
api_key = ''
api_key_test = ''
elabftw_api = ElabFTWAPI(base_url, api_key)

In [ ]:
elabftw_api.get_item(50)

{'access_key': None,
 'available': 1,
 'body': '<p>This is the default text of the default category.</p>\n<p>Head to the <a href="admin.php?tab=5">Admin Panel</a> to edit/add more categories for your database!</p>',
 'body_html': '<p>This is the default text of the default category.</p>\n<p>Head to the <a href="admin.php?tab=5">Admin Panel</a> to edit/add more categories for your database!</p>',
 'book_can_overlap': 1,
 'book_cancel_minutes': 0,
 'book_is_cancellable': 1,
 'book_max_minutes': 0,
 'book_max_slots': 0,
 'book_users_can_in_past': 0,
 'canbook': '{"base": 30, "teams": [], "users": [], "teamgroups": []}',
 'canread': '{"base": 30, "teams": [], "users": [], "teamgroups": []}',
 'canwrite': '{"base": 30, "teams": [], "users": [], "teamgroups": []}',
 'category': 33,
 'category_color': '32a100',
 'category_title': 'Samples',
 'comments': [{'id': 4,
   'created_at': '2025-03-12 10:42:12',
   'modified_at': '2025-03-12 10:42:12',
   'item_id': 50,
   'comment': 'Observations: Th

In [ ]:
import copy
import json

class SampleMetadata:
    def __init__(self, data):
        self._original = copy.deepcopy(data)
        self._fields = {}  # editable fields only

        for field_name, props in data.get("extra_fields", {}).items():
            value = props.get("value", None)
            if value is not None:
                attr_name = self._normalize_name(field_name)
                self._fields[attr_name] = value
                setattr(self, attr_name, value)

    def _normalize_name(self, name):
        return name.strip().lower().replace(" ", "_").replace("-", "_").replace("(", "").replace(")", "")

    def to_json(self):
        updated = copy.deepcopy(self._original)
        for field_name, props in updated.get("extra_fields", {}).items():
            attr_name = self._normalize_name(field_name)
            if hasattr(self, attr_name):
                props["value"] = getattr(self, attr_name)
        return updated

    def __str__(self):
        return json.dumps(self.to_json(), indent=2)

    def list_editable_fields(self):
        return list(self._fields.keys())




item_json_string = """{
    "elabftw": {
        "extra_fields_groups": [
            {
                "id": 1,
                "name": "Sample identification"
            },
            {
                "id": 2,
                "name": "Sample Description"
            },
            {
                "id": 3,
                "name": "Sample characterization"
            },
            {
                "id": 4,
                "name": "Features of interest"
            },
            {
                "id": 5,
                "name": "Sample Handling Precaution"
            },
            {
                "id": 6,
                "name": "Sample Preparation"
            },
            {
                "id": 7,
                "name": "Sample Holder"
            },
            {
                "id": 8,
                "name": "Grids"
            }
        ]
    },
    "extra_fields": {
        "Other": {
            "type": "text",
            "value": "",
            "group_id": 4,
            "position": 4,
            "description": "Comment"
        },
        "Sheet": {
            "type": "radio",
            "value": "not applicable",
            "options": [
                "not applicable",
                "foil",
                "plate",
                "leaf"
            ],
            "group_id": 2,
            "position": 3,
            "description": "Sheet type"
        },
        "Gloves": {
            "type": "checkbox",
            "value": "on",
            "group_id": 5,
            "position": 0
        },
        "Layer ": {
            "type": "radio",
            "value": "not applicable",
            "options": [
                "not applicable",
                "monolayer",
                "thin film",
                "multilayer"
            ],
            "group_id": 2,
            "position": 4,
            "description": "Layer type"
        },
        "Twinned": {
            "type": "checkbox",
            "value": "",
            "group_id": 4,
            "position": 1,
            "description": "Two separate crystal domains share some of the same crystal lattice points in a symmetrical manner"
        },
        "Mesh grid": {
            "type": "number",
            "unit": "",
            "units": [],
            "value": "400",
            "group_id": 8,
            "position": 4
        },
        "Particles": {
            "type": "radio",
            "value": "liquid (droplets)",
            "options": [
                "not applicable",
                "solid (fleks)",
                "liquid (droplets)",
                "gaseous (bubbles)"
            ],
            "group_id": 4,
            "position": 8,
            "description": "Particle type"
        },
        "Sample_ID": {
            "type": "text",
            "value": "004",
            "group_id": 1,
            "position": 5
        },
        "User name": {
            "type": "text",
            "value": "Daniela Manno",
            "group_id": 6,
            "position": 0,
            "required": true
        },
        "User role": {
            "type": "radio",
            "value": "Instrument scientist",
            "options": [
                "Data curator",
                "Instrument scientist",
                "Team leader",
                "Team member"
            ],
            "group_id": 6,
            "position": 1
        },
        "Interfaces": {
            "type": "select",
            "value": [
                "none"
            ],
            "options": [
                "none",
                "antiphase boundaries",
                "grain boundaries",
                "magnetic domain walls",
                "matrix-fiber interfaces",
                "matrix-particle interfaces",
                "phase boundaries",
                "stacking faults",
                "surfaces",
                "twin boundaries",
                "other (please specify in the comment)"
            ],
            "group_id": 4,
            "position": 5,
            "description": "Multiple choice is allowed",
            "allow_multi_values": true
        },
        "Affiliation": {
            "type": "text",
            "value": "University of Salento",
            "group_id": 1,
            "position": 2,
            "required": true
        },
        "Sample Name": {
            "type": "text",
            "value": "Sample 004",
            "group_id": 1,
            "position": 0,
            "required": true
        },
        "Sample type": {
            "type": "radio",
            "value": "nano",
            "options": [
                "nano",
                "bulk/film",
                "bio"
            ],
            "group_id": 2,
            "position": 0,
            "required": true
        },
        "Crystallinity": {
            "type": "select",
            "value": "atomic/molecular",
            "options": [
                "not applicable",
                "atomic/molecular",
                "nanoscopic",
                "microscopic",
                "mesoscopic",
                "macroscopic"
            ],
            "group_id": 4,
            "position": 2,
            "description": "Scale"
        },
        "Component name": {
            "type": "text",
            "value": "urea and citric acid",
            "group_id": 2,
            "position": 1
        },
        "Gas atmosphere": {
            "type": "select",
            "value": "air",
            "options": [
                "not applicable",
                "air",
                "dry air",
                "vacuum",
                "Ar",
                "N",
                "other (please add in the comment)"
            ],
            "group_id": 5,
            "position": 1,
            "description": "Type of inert gas required around the sample to handle it, due e.g. to the presence of a reactive top layer"
        },
        "Sample Purpose": {
            "type": "select",
            "value": [
                "optimization of parameters/properties"
            ],
            "options": [
                "feasibility check (quick and dirty, rough estimate)",
                "parameter calibration",
                "optimization of parameters/properties",
                "fullfilment of given requirements",
                "assessment (check presence or absence of given properties)",
                "investigation of properties (which are unknown before measurements)",
                "test of a specific hypothesis (focus only on given aspects)",
                "correlative characterization (dedicated sample treatment to emphasise given features)",
                "high quality sample (precise, careful treatment)",
                "further preparations",
                "other (please specify in the comment)"
            ],
            "group_id": 1,
            "position": 3,
            "description": "Multiple selection is allowed",
            "allow_multi_values": true
        },
        "Sample_ID Type": {
            "type": "select",
            "value": "value",
            "options": [
                "not applicable",
                "text",
                "code",
                "value",
                "other (please specify in the comment)"
            ],
            "group_id": 1,
            "position": 6
        },
        "Other Sample_ID": {
            "type": "text",
            "value": "",
            "group_id": 1
        },
        "Particles shape": {
            "type": "text",
            "value": "heterogeneous mixture of nanostructures. Some regions contain well-defined crystalline nanoparticles, while others exhibit amorphous or poorly structured particles. ",
            "group_id": 4,
            "position": 9
        },
        "Sample Producer": {
            "type": "text",
            "value": "Daniela Manno",
            "group_id": 1,
            "position": 1,
            "required": true,
            "description": "Name of the scientist"
        },
        "Additional notes": {
            "type": "text",
            "value": "",
            "group_id": 5,
            "position": 10,
            "description": "Any additional notes which might be relevant for sample handling"
        },
        "Shock protection": {
            "type": "checkbox",
            "value": "",
            "group_id": 5,
            "position": 2,
            "description": "The sample must be protected against shocks"
        },
        "Colloidal solution": {
            "type": "text",
            "value": "citric acid and urea were dissolved in distilled water and the prepared solution was heated in a microwave; then, filtered",
            "group_id": 2
        },
        "Crystal structures": {
            "type": "select",
            "value": "nano-crystalline",
            "options": [
                "not applicable",
                "nano-crystalline",
                "crystalline/single crystal",
                "polycrystalline",
                "semicrystalline",
                "non-crystalline/amorphous"
            ],
            "group_id": 4,
            "position": 0,
            "description": "Crystallinity"
        },
        "Safety information": {
            "type": "select",
            "value": "nanostructured/nanoparticles",
            "options": [
                "reactive",
                "radioactive",
                "oxidising",
                "corrosive",
                "contaminant",
                "combustive",
                "biohazard",
                "carcinogenic/mutagenic/teratogenic",
                "inflammable",
                "toxic or irritant",
                "explosive",
                "nanostructured/nanoparticles",
                "other (please add in the comments)"
            ],
            "group_id": 5,
            "position": 5,
            "description": "Hazard"
        },
        "Sample holder type": {
            "type": "radio",
            "value": "EM-21311HTR-HIGH TILT SAMPLE HOLDER                                       ",
            "options": [
                "EM- 01350RSTHB- REINFORCED SPECIMEN TILTING BERYLLIUM HOLDER",
                "EM-21311HTR-HIGH TILT SAMPLE HOLDER                                       ",
                "GATAN® 915 CRYO HOLDER"
            ],
            "group_id": 7
        },
        "Lamellar structures": {
            "type": "select",
            "value": "not applicable",
            "options": [
                "not applicable",
                "atomic/molecular",
                "nanoscopic",
                "microscopic",
                "mesoscopic",
                "macroscopic"
            ],
            "group_id": 4,
            "position": 7,
            "description": "Scale"
        },
        "Sensitivity against": {
            "type": "select",
            "value": "eBeam",
            "options": [
                "O2",
                "H2O/moisture",
                "N2",
                "organic solvents",
                "eBeam",
                "iBeam",
                "radiation",
                "shocks",
                "variations of temperature",
                "variations of air pressure",
                "variations of gas concentration",
                "variations of humidity",
                "other (please add in the comment)"
            ],
            "group_id": 5,
            "position": 3,
            "description": "The sample is sensitive to"
        },
        "Other Sample Purpose": {
            "type": "text",
            "value": "optimization of synthesis parameters to ensure maximum reproducibility.",
            "group_id": 1,
            "position": 4
        },
        "Other gas atmosphere": {
            "type": "text",
            "value": "",
            "group_id": 5,
            "position": 9
        },
        "Support film on grid": {
            "type": "select",
            "value": "Lacey Carbon",
            "options": [
                "Formvar",
                "Carbon",
                "Formvar + Carbon",
                "Lacey Carbon",
                "Lacey Formvar Carbon Grids",
                "Holey Carbon ",
                "Other"
            ],
            "group_id": 8,
            "position": 2
        },
        "Clean room conditions": {
            "type": "text",
            "value": "",
            "group_id": 5,
            "position": 8,
            "description": "Specific clean room conditions under which the sample should be treated"
        },
        "Material type of grid": {
            "type": "select",
            "value": "Copper",
            "options": [
                "Copper ",
                "Nickel",
                "Gold",
                "Molybdenum",
                "Titanium",
                "Other"
            ],
            "group_id": 8,
            "position": 0
        },
        "Nanosheets- thickness": {
            "type": "text",
            "value": "",
            "group_id": 4,
            "position": 12,
            "description": "Value/unit"
        },
        "Sample expiration date": {
            "type": "date",
            "value": "",
            "group_id": 2
        },
        "Defects (if applicable)": {
            "type": "select",
            "value": [
                "none"
            ],
            "options": [
                "none",
                "cracks",
                "crazes",
                "inclusions",
                "pores",
                "voids",
                "dislocations",
                "antisite defects",
                "interstitial defects",
                "topological defects",
                "vacancies",
                "other (please specify in the comment)"
            ],
            "group_id": 4,
            "position": 3,
            "description": "Multiple choice is allowed",
            "allow_multi_values": true
        },
        "Sample preparation date": {
            "type": "date",
            "value": "",
            "group_id": 6,
            "position": 2
        },
        "Sample preparation file": {
            "type": "radio",
            "value": "paper log book",
            "options": [
                "not applicable",
                "ELN",
                "data repository",
                "included in the raw data file",
                "paper log book"
            ],
            "group_id": 6,
            "position": 6
        },
        "Other type of interfaces": {
            "type": "text",
            "value": "",
            "group_id": 4,
            "position": 6
        },
        "Type of characterization": {
            "type": "select",
            "value": [
                "TEM",
                "HRTEM",
                "STEM"
            ],
            "options": [
                "TEM",
                "HRTEM",
                "STEM",
                "SAD",
                "EELS",
                "EDS",
                "TOMO",
                "HOLO",
                "Other"
            ],
            "group_id": 3,
            "allow_multi_values": true
        },
        "Sample preparation action": {
            "type": "select",
            "value": "drop-casting",
            "options": [
                "not applicable",
                "drop-casting",
                "annealing homogenization",
                "deposition coating",
                "joining",
                "mechanical and surface",
                "powder processing",
                "cooling",
                "reactive",
                "other (please add in the comments)"
            ],
            "group_id": 6,
            "position": 3
        },
        "Component chemical formula": {
            "type": "text",
            "value": "CH4N2O and C6H8O7 ",
            "group_id": 2,
            "position": 2
        },
        "Other support film on grid": {
            "type": "text",
            "value": "",
            "group_id": 8,
            "position": 3
        },
        "Safety information (other)": {
            "type": "text",
            "value": "",
            "group_id": 5,
            "position": 6,
            "description": "Comment"
        },
        "Material type of the sample": {
            "type": "select",
            "value": "colloidal",
            "options": [
                "not applicable",
                "alloy",
                "biological",
                "biomaterial",
                "ceramic",
                "composite",
                "glass",
                "metal",
                "metamaterial",
                "molecular fluid",
                "organic compound",
                "organometallic",
                "polymer",
                "smart material",
                "colloidal"
            ],
            "group_id": 3,
            "required": true
        },
        "Other material type of grid": {
            "type": "text",
            "value": "",
            "group_id": 8,
            "position": 1
        },
        "Sensitivity against (other)": {
            "type": "text",
            "value": "",
            "group_id": 5,
            "position": 4,
            "description": "Comment"
        },
        "Particles- Max particle size": {
            "type": "text",
            "value": "5nm",
            "group_id": 4,
            "position": 11,
            "description": "Value/unit"
        },
        "Particles- Min particle size": {
            "type": "text",
            "value": "2nm",
            "group_id": 4,
            "position": 10,
            "description": "Value/unit"
        },
        "Phase of matter of the sample": {
            "type": "select",
            "value": "liquid",
            "options": [
                "not applicable",
                "solid",
                "liquid",
                "gas",
                "plasma",
                "mixture"
            ],
            "group_id": 3
        },
        "Sample preparation description": {
            "type": "text",
            "value": " citric acid and urea were dissolved in distilled water, and the prepared solution was heated in a microwave oven at a power of 540W for 10 minutes (180°<T<220°), reaching a local temperature of 300°C. The total volume of the solution reached 5ml. By the end of the process, the solution had turned black. It was then diluited in water and subjected to an ultrasonic bath for 15 minutes until the product was fully homogenized.",
            "group_id": 6,
            "position": 5
        },
        "Material properties of the sample": {
            "type": "select",
            "value": [
                "other (please add in the comment)"
            ],
            "options": [
                "diamagnetic",
                "paramagnetic",
                "ferromagnetic",
                "antiferromagnetic",
                "ferrimagnetic",
                "nonmagnetic",
                "conductor",
                "semiconductor",
                "superconductor",
                "insulator",
                "dielectric",
                "bio",
                "other (please add in the comment)"
            ],
            "group_id": 3,
            "allow_multi_values": true
        },
        "Sample preparation action (other)": {
            "type": "text",
            "value": "",
            "group_id": 6,
            "position": 4,
            "description": "Comments"
        },
        "Sample preparation file reference": {
            "type": "text",
            "value": "",
            "group_id": 6
        },
        "Sample handling- humidity (value/unit)": {
            "type": "text",
            "value": "",
            "group_id": 5,
            "position": 7
        },
        "Other material properties of the sample": {
            "type": "text",
            "value": "Fluorescent",
            "group_id": 3
        }
    }
}"""

In [ ]:
# Assuming item_json_string is your big JSON string
item_data = json.loads(item_json_string)

sample = SampleMetadata(item_data)

# Show editable fields
print(sample.list_editable_fields())  # Example output: ['sample_id', 'user_name', 'affiliation', ...]

# Modify some fields
sample.sample_id = "002"
sample.user_name = "Antonio Serra"
sample.affiliation =  "Unisalento"

# Get updated JSON
#print(sample)


['other', 'sheet', 'gloves', 'layer', 'twinned', 'mesh_grid', 'particles', 'sample_id', 'user_name', 'user_role', 'interfaces', 'affiliation', 'sample_name', 'sample_type', 'crystallinity', 'component_name', 'gas_atmosphere', 'sample_purpose', 'sample_id_type', 'other_sample_id', 'particles_shape', 'sample_producer', 'additional_notes', 'shock_protection', 'colloidal_solution', 'crystal_structures', 'safety_information', 'sample_holder_type', 'lamellar_structures', 'sensitivity_against', 'other_sample_purpose', 'other_gas_atmosphere', 'support_film_on_grid', 'clean_room_conditions', 'material_type_of_grid', 'nanosheets__thickness', 'sample_expiration_date', 'defects_if_applicable', 'sample_preparation_date', 'sample_preparation_file', 'other_type_of_interfaces', 'type_of_characterization', 'sample_preparation_action', 'component_chemical_formula', 'other_support_film_on_grid', 'safety_information_other', 'material_type_of_the_sample', 'other_material_type_of_grid', 'sensitivity_against

In [ ]:
# Example usage for table of metadata in main text:

body_html = generate_body_html(Goal="to produce a fluorescent Sample", Synthesis_Parameters= "", Results="Sample 002", )
metadata = { "": ''}
body_html += generate_html_table(metadata, title="Metadata")
print(body_html)



# Example usage of create experiment (replace with your actual data)

new_items_data = build_item_payload( title='Fluorescent C-dots (Sample002)',
    body_html=body_html,
    date='2025-03-23',
    category=1,
    status=2,
    metadata= json.dumps(sample.to_json())
)
# print(new_experiment_data)



new_item = elabftw_api.create_item(new_items_data,tags=['carbondots','fluorescent c-dots','nanoparticles','synthesis'])
print(json.dumps(new_item, indent=2))


<p><strong>Goal:</strong> to produce a fluorescent Sample</p>
<p><strong>Synthesis parameters:</strong> </p>
<p><strong>Results:</strong> Sample 002</p><p><strong>Metadata</strong></p>
<table border='1' cellspacing='0' cellpadding='5'><thead><tr><th>Parameter</th><th>Value</th></tr></thead><tbody><tr><td></td><td></td></tr></tbody></table>
{'tag': 'carbondots'}
{'tag': 'fluorescent c-dots'}
{'tag': 'nanoparticles'}
{'tag': 'synthesis'}
{
  "access_key": null,
  "available": 1,
  "body": "<p><strong>Goal:</strong> to produce a fluorescent Sample</p>\n<p><strong>Synthesis parameters:</strong> </p>\n<p><strong>Results:</strong> Sample 002</p><p><strong>Metadata</strong></p>\n<table border=\"1\"><tr><th>Parameter</th><th>Value</th></tr><tr><td></td><td></td></tr></table>",
  "body_html": "<p><strong>Goal:</strong> to produce a fluorescent Sample</p>\n<p><strong>Synthesis parameters:</strong> </p>\n<p><strong>Results:</strong> Sample 002</p><p><strong>Metadata</strong></p>\n<table border=\"